In [8]:
import tensorflow as tf

In [62]:
# Do not Touch!
class BasicBlock(tf.keras.layers.Layer):
    def __init__(self, in_filters, out_filters, downsample=False):
        super(BasicBlock, self).__init__()
        self.pad1 = tf.keras.layers.ZeroPadding2D((1,1))

        self.relu = tf.keras.layers.ReLU()
        self.conv2 = tf.keras.layers.Conv2D(out_filters, (3,3), strides=(1,1))
        
        self.batch_norm1 = tf.keras.layers.BatchNormalization()
        self.batch_norm2 = tf.keras.layers.BatchNormalization()

        if downsample:
            self.conv1 = tf.keras.layers.Conv2D(out_filters, (3,3), strides=(2,2))
            self.downsample = tf.keras.layers.Conv2D(out_filters, (1,1), strides=(2,2))
        else:
            self.conv1 = tf.keras.layers.Conv2D(out_filters, (3,3), strides=(1,1))
            self.downsample = None

        if in_filters == out_filters:
            self.channel_shaper = None
        else:
            self.channel_shaper = tf.keras.layers.Conv2D(out_filters, (1,1), strides=(1,1))


    def __call__(self, inputs):
        identity = inputs
        out = self.pad1(inputs)
        out = self.conv1(out)
        out = self.batch_norm1(out)
        out = self.relu(out)
        out = self.pad1(out)
        out = self.conv2(out)
        out = self.batch_norm2(out)
        if self.downsample is not None:
            identity = self.downsample(inputs)
        if self.channel_shaper is not None:
            identity = self.channel_shaper(identity)

        out += identity 
        out = self.relu(out)
        return out

In [74]:
class ResNet18(tf.keras.Model):
    def __init__(self, num_classes) -> None:
        super(ResNet18, self).__init__()
        self.pad3 = tf.keras.layers.ZeroPadding2D((3,3))
        self.pad1 = tf.keras.layers.ZeroPadding2D((1,1))
        self.conv1 = tf.keras.layers.Conv2D(64, kernel_size=(7,7), strides=(2,2))
        self.pool = tf.keras.layers.MaxPool2D(pool_size=(3,3), strides=(2,2))
        self.batch_norm = tf.keras.layers.BatchNormalization()
        self.relu = tf.keras.layers.ReLU()

        # 문제 1: BasicBlock을 활용하여, Convolution층을 정의하세요.
        # Your Code Here!

        # build_layer 함수를 통해서 각각 두번씩 BasicBlock을 쌓습니다.
        # block b에서는 샘플의 크기가 변하지 않고, block c, d, e에서만 절반으로 줄어들어야 하기 때문에
        # conv라는 옵션을 통해 downsampling block 추가 유무를 결정합니다.        
        self.block_b = self.build_layers(64, 64, 2, conv=False)
        self.block_c = self.build_layers(64, 128, 2, conv=True)
        self.block_d = self.build_layers(128, 256, 2, conv=True)
        self.block_e = self.build_layers(256, 512, 2, conv=True)

        # 마지막 단계에서 Global Average Pooling 실시
        self.Avgpooling = tf.keras.layers.GlobalAveragePooling2D()
        # Fully connected layer형성을 위한 flatten()
        self.flat = tf.keras.layers.Flatten()
        # num_class수 만큼의 output 반환. 활성화 함수 softmax사용
        self.dense = tf.keras.layers.Dense(num_classes, activation="softmax")

    def build_layers(self, in_filters, out_filters, num_blocks, conv=False):
        # layer 리스트에 필요한 layer들을 append한 다음 한번에 tf.keras.Sequential로 레이어를 반환합니다.
        layers= []
        # conv 옵션이 활성화 될 경우에는 downsampling=True를 가장 첫 BasicBlock에서 활성시켜야 합니다.
        # 활성화 될 시 stride의 크기가 2로 설정되어 샘플의 크기가 절반으로 줄어듭니다.
        # 첫 BasicBlock의 경우 필터 크기의 변환도 실행해야 하기 때문에, in_filters와 out_filters를 입력값으로 줍니다.
        if conv :
            layers.append(BasicBlock(in_filters, out_filters, downsample=True))
        else : 
            layers.append(BasicBlock(in_filters, out_filters, downsample=False))
        # 두번째 블록부터는 in_filter와 out_filter의 크기가 같습니다.
        # 따라서 out_filter값을 in_filter, out_filter에 넣어서 BasicBlock을 쌓습니다.
        for i in range(num_blocks-1):
            layers.append(BasicBlock(out_filters, out_filters, downsample=False))

        # 쌓인 레이어들을 한번에 Sequential을 통해 반환합니다.
        return tf.keras.Sequential(layers)

    def call(self, x):
        # 문제 2: 모델의 레이어를 정의하세요.
        # Your Code Here!
        out = self.conv1(x) # kernel size 7,7, strides=2, 샘플 크기가 절반으로 줄어듭니다. 64개의 채널을 가집니다.
        out = self.pad3(out) # kernel size 3,3, zero padding
        out = self.batch_norm(out)
        out = self.relu(out)
        out = self.pool(out) # kenrel size 3,3 strides=2, 샘플 크기가 절반으로 줄어듭니다.
        out = self.pad1(out) # kernel size 1,1 로 zero padding

        out = self.block_b(out) # input channel 수 64, output channel 수 64
        out = self.block_c(out) # input channel 수 64, output channel 수 128
        out = self.block_d(out) # input channel 수 128, output channel 수 256
        out = self.block_e(out) # input channel 수 256, output channel 수 512

        out = self.Avgpooling(out) # global average pooling 
        out = self.flat(out) # flatten layer
        out = self.dense(out) # Fully connected layer형성, num_class수 만큼의 output layer반환. 활성화 함수 softmax사용
        return out


In [75]:
# Do not Touch!
model = ResNet18(num_classes=10)
model.build(input_shape=[None, 28,28,3])
model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

Model: "res_net18_30"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d_288 (ZeroPad  multiple                 0         
 ding2D)                                                         
                                                                 
 zero_padding2d_289 (ZeroPad  multiple                 0         
 ding2D)                                                         
                                                                 
 conv2d_666 (Conv2D)         multiple                  9472      
                                                                 
 max_pooling2d_30 (MaxPoolin  multiple                 0         
 g2D)                                                            
                                                                 
 batch_normalization_486 (Ba  multiple                 256       
 tchNormalization)                                    

In [76]:
# Do not Touch!
# Loss Function을 변수로 정의
import tensorflow as tf

loss_function = tf.keras.losses.CategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

train_acc = tf.keras.metrics.CategoricalAccuracy()
test_acc = tf.keras.metrics.CategoricalAccuracy() 
val_acc = tf.keras.metrics.CategoricalAccuracy() 

train_loss = tf.keras.metrics.Mean()
test_loss = tf.keras.metrics.Mean()
val_loss = tf.keras.metrics.Mean()

In [77]:
# Do not Touch!
@tf.function
def train_step(images, labels):

    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_function(labels, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss.update_state(loss)
    train_acc.update_state(labels, predictions)

In [78]:
# Do not Touch!
@tf.function
def test_step(images, labels):
    predictions = model(images)
    loss = loss_function(labels, predictions)
    
    test_loss.update_state(loss)
    test_acc.update_state(labels, predictions)

    return labels, predictions

In [79]:
# Do not Touch!
@tf.function
def val_step(images, labels):
    predictions = model(images)
    loss = loss_function(labels, predictions)
    
    val_loss.update_state(loss)
    val_acc.update_state(labels, predictions)

In [80]:
# Do not Touch!
import math 

class generator(tf.keras.utils.Sequence):
    def __init__(self, x, y, batch_size, shuffle =True):
        self.x = x.astype(np.float32)
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def on_epoch_end(self):
        self.indices = np.arange(len(self.x))
        if self.shuffle == True:
            np.random.shuffle(self.indices)

    def __getitem__(self, index):
        indices = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        return self.x[indices], self.y[indices]

In [55]:
from tensorflow.keras.datasets import cifar10
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
import numpy as np

# CIFAR-10 데이터셋 불러오기
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()

# 문제 3 (Optional): 정확도를 올리기 위한 데이터 전처리를 수행하셔도 됩니다.

# 레이블을 범주형으로 인코딩
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

170498071/170498071 [==============================] - 633s 4us/step


In [81]:
# Do not Touch!
from sklearn.model_selection import train_test_split

train_images, val_images, train_labels, val_labels = train_test_split(train_images, train_labels, test_size=0.2, random_state=10)

train_ds = generator(train_images, train_labels, batch_size=32)
val_ds = generator(val_images, val_labels, batch_size=32)
test_ds = generator(test_images, test_labels, batch_size=32, shuffle=False)

In [82]:
# Do not Touch!
from tqdm import tqdm
EPOCHS = 10
min_loss = float("inf")
for epoch in range(EPOCHS):
    print(f"Epoch{epoch} Training")
    for images, labels in tqdm(train_ds):
        train_step(images, labels)
        
    for test_images, test_labels in tqdm(val_ds):
        val_step(test_images, test_labels)

    template = '에포크: {}, 손실: {:.5f}, 정확도: {:.2f}%, 테스트 손실: {:.5f}, 테스트 정확도: {:.2f}%'
    print (template.format(epoch+1,
                           train_loss.result(),
                           train_acc.result()*100,
                           val_loss.result(),
                           val_acc.result()*100))
    
    if val_loss.result() < min_loss:
        model.save_weights("./model.h5")
        print("✅ Model Loss Updated")
        min_loss = val_loss.result()
    
    train_loss.reset_states()
    train_acc.reset_states()
    val_loss.reset_states()
    val_acc.reset_states()

Epoch0 Training


100%|██████████| 200/200 [00:04<00:00, 47.61it/s]


에포크: 1, 손실: 2.25666, 정확도: 30.91%, 테스트 손실: 1.65182, 테스트 정확도: 38.28%
✅ Model Loss Updated
Epoch1 Training


100%|██████████| 200/200 [00:03<00:00, 53.01it/s]


에포크: 2, 손실: 1.56491, 정확도: 43.02%, 테스트 손실: 1.49243, 테스트 정확도: 45.38%
✅ Model Loss Updated
Epoch2 Training


...

에포크: 8, 손실: 1.04227, 정확도: 63.03%, 테스트 손실: 1.48696, 테스트 정확도: 52.25%
Epoch8 Training


100%|██████████| 200/200 [00:03<00:00, 53.83it/s]


에포크: 9, 손실: 0.97591, 정확도: 65.54%, 테스트 손실: 1.65052, 테스트 정확도: 49.73%
Epoch9 Training


100%|██████████| 200/200 [00:03<00:00, 57.87it/s]

에포크: 10, 손실: 0.91895, 정확도: 67.34%, 테스트 손실: 1.60403, 테스트 정확도: 54.12%


In [83]:
# Do not Touch!
for test_images, test_labels in tqdm(test_ds):
    test_step(test_images, test_labels)

100%|██████████| 1/1 [00:01<00:00,  1.38s/it]


In [84]:
# Do not Touch!
print(f"Test Accuracy: {test_acc.result()*100:.2f}%")

Test Accuracy: 56.25%
